In [9]:
getwd()
if ( is.null(environment()$this_notebook_dir) ) {
    this_notebook_dir <- getwd()
    setwd(paste0(getwd(), '/..'))
}
getwd()

[1] "/mnt/hdd/develop/kaggle/house-prices/R"

[1] "/mnt/hdd/develop/kaggle/house-prices/R"

In [10]:
source('helpers.R')

kaggle.house.loadLibraries()
data <- kaggle.house.loadData()

data$combined %>% 
select(-dataSource) %>% 
filter(!is.na(LotFrontage)) %>% 
mutate(
    LotFrontage.log = log(LotFrontage),
    LotArea.log = log(LotArea)
) -> df.LotFrontage

df.LotFrontage %>% dim

[1] 2433   81

In [42]:
df.LotFrontage %>% 
lm(LotFrontage.log ~ LotArea.log, data = .) -> model

model %>% summary
sum(model$residuals ^ 2)


Call:
lm(formula = LotFrontage.log ~ LotArea.log, data = .)

Residuals:
     Min       1Q   Median       3Q      Max 
-1.33420 -0.12683  0.02732  0.13909  1.25960 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) -0.816932   0.085101    -9.6   <2e-16 ***
LotArea.log  0.551207   0.009374    58.8   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.2299 on 2431 degrees of freedom
Multiple R-squared:  0.5872,	Adjusted R-squared:  0.587 
F-statistic:  3458 on 1 and 2431 DF,  p-value: < 2.2e-16


[1] 128.4953

In [32]:
df.LotFrontage %>% 
select(LotFrontage.log, LotArea.log, MSSubClass) %>%
lm(LotFrontage.log ~ LotArea.log + MSSubClass, data = .) -> model

model %>% summary
sum(model$residuals ^ 2)


Call:
lm(formula = LotFrontage.log ~ LotArea.log + MSSubClass, data = .)

Residuals:
     Min       1Q   Median       3Q      Max 
-1.02965 -0.09845  0.00116  0.10641  1.14375 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)    0.87696    0.10829   8.098 8.75e-16 ***
LotArea.log    0.33748    0.01253  26.935  < 2e-16 ***
MSSubClass160 -0.24130    0.02740  -8.806  < 2e-16 ***
MSSubClass180 -0.24097    0.05546  -4.345 1.45e-05 ***
MSSubClass190  0.22774    0.03308   6.884 7.39e-12 ***
MSSubClass20   0.32685    0.02032  16.082  < 2e-16 ***
MSSubClass30   0.20083    0.02545   7.892 4.48e-15 ***
MSSubClass40   0.04072    0.09436   0.431   0.6661    
MSSubClass45   0.15438    0.05178   2.981   0.0029 ** 
MSSubClass50   0.19014    0.02214   8.589  < 2e-16 ***
MSSubClass60   0.32849    0.02171  15.128  < 2e-16 ***
MSSubClass70   0.19911    0.02635   7.556 5.86e-14 ***
MSSubClass75   0.27565    0.04897   5.629 2.02e-08 ***
MSSubClass80   0.34972    0.02940  11

[1] 103.6882

In [23]:
df.LotFrontage %>% select(MSSubClass, LotFrontage.log) %>%
group_by(MSSubClass) %>%
summarise(n = n(), median = median(LotFrontage.log)) %>%
mutate(freq = (n / sum(n)) * 100) %>%
arrange(median)

MSSubClass,n,median,freq
180,16,3.044522,0.6576243
160,116,3.178054,4.7677764
120,150,3.761200,6.1652281
40,5,4.007333,0.2055076
45,18,4.007333,0.7398274
190,57,4.094345,2.3427867
30,126,4.094345,5.1787916
50,262,4.094345,10.7685984
70,117,4.094345,4.8088779
75,21,4.174387,0.8631319


In [46]:
df <- df.LotFrontage %>%
mutate(
    MSSubClass2 = case_when(
        MSSubClass == '20' ~ 'A',
        MSSubClass %in% c('60', '70', '75', '80', '85', '90') ~ 'B',
        TRUE ~ 'C'
)) 

df %>%
select(MSSubClass2, MSSubClass, LotFrontage.log) %>%
group_by(MSSubClass2) %>%
summarise(n = n(), median = median(LotFrontage.log))

df %>% 
lm(LotFrontage.log ~ LotArea.log + MSSubClass2, data = .) -> model

model %>% summary
sum(model$residuals ^ 2)

MSSubClass2,n,median
A,894,4.317488
B,789,4.290459
C,750,3.951244



Call:
lm(formula = LotFrontage.log ~ LotArea.log + MSSubClass2, data = .)

Residuals:
     Min       1Q   Median       3Q      Max 
-1.23635 -0.10151  0.01983  0.11829  1.31626 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept)  -0.06416    0.09627  -0.667    0.505    
LotArea.log   0.47461    0.01038  45.722   <2e-16 ***
MSSubClass2B -0.01665    0.01077  -1.546    0.122    
MSSubClass2C -0.17189    0.01233 -13.936   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.2204 on 2429 degrees of freedom
Multiple R-squared:  0.6211,	Adjusted R-squared:  0.6206 
F-statistic:  1327 on 3 and 2429 DF,  p-value: < 2.2e-16


[1] 117.9403

In [47]:
df <- df.LotFrontage %>%
group_by(MSSubClass) %>%
mutate(MSSubClass.avg = median(LotFrontage.log))

df %>% 
lm(LotFrontage.log ~ LotArea.log + MSSubClass.avg, data = .) -> model

model %>% summary
sum(model$residuals ^ 2)


Call:
lm(formula = LotFrontage.log ~ LotArea.log + MSSubClass.avg, 
    data = .)

Residuals:
     Min       1Q   Median       3Q      Max 
-1.02247 -0.10112  0.00395  0.10651  1.13561 

Coefficients:
               Estimate Std. Error t value Pr(>|t|)    
(Intercept)    -1.01192    0.07714  -13.12   <2e-16 ***
LotArea.log     0.33655    0.01238   27.18   <2e-16 ***
MSSubClass.avg  0.51427    0.02168   23.72   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.2072 on 2430 degrees of freedom
Multiple R-squared:  0.6648,	Adjusted R-squared:  0.6645 
F-statistic:  2410 on 2 and 2430 DF,  p-value: < 2.2e-16


[1] 104.3419

In [54]:
df <- df.LotFrontage %>%
select(LotFrontage.log, LotArea.log, MSSubClass) %>%
mutate(
    MSSubClass2 = case_when(
        MSSubClass == '20' ~ 'A',
        MSSubClass %in% c('60', '70', '75', '80', '85', '90') ~ 'B',
        TRUE ~ 'C'
)) %>%
group_by(MSSubClass) %>%
mutate(
    MSSubClass.avg = median(LotFrontage.log),
    MSSubClass.A = ifelse(MSSubClass2 == 'A', MSSubClass.avg, 0),
    MSSubClass.B = ifelse(MSSubClass2 == 'B', MSSubClass.avg, 0),
    MSSubClass.C = ifelse(MSSubClass2 == 'C', MSSubClass.avg, 0)
) 

df %>% 
lm(LotFrontage.log ~ LotArea.log + MSSubClass.A + MSSubClass.B + MSSubClass.C, data = .) -> model

model %>% summary
sum(model$residuals ^ 2)


Call:
lm(formula = LotFrontage.log ~ LotArea.log + MSSubClass.A + MSSubClass.B + 
    MSSubClass.C, data = .)

Residuals:
     Min       1Q   Median       3Q      Max 
-1.02793 -0.09510  0.00171  0.10500  1.15389 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept)  -0.89727    0.09314  -9.634   <2e-16 ***
LotArea.log   0.33770    0.01239  27.257   <2e-16 ***
MSSubClass.A  0.48618    0.02516  19.325   <2e-16 ***
MSSubClass.B  0.48608    0.02535  19.178   <2e-16 ***
MSSubClass.C  0.47947    0.02689  17.833   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.2071 on 2428 degrees of freedom
Multiple R-squared:  0.6655,	Adjusted R-squared:  0.6649 
F-statistic:  1207 on 4 and 2428 DF,  p-value: < 2.2e-16


[1] 104.1341

# 2

In [70]:
df <- df.LotFrontage %>%
select(LotFrontage.log, LotArea.log, MSSubClass, Neighborhood) %>% 
lm(LotFrontage.log ~ LotArea.log + MSSubClass + Neighborhood, data = .) -> model

#model %>% summary
sum(model$residuals ^ 2)

[1] 94.15694

In [59]:
df <- df.LotFrontage %>%
select(LotFrontage.log, LotArea.log, MSSubClass, Neighborhood) %>%
group_by(MSSubClass) %>%
mutate(MSSubClass.avg = median(LotFrontage.log))

df %>% 
lm(LotFrontage.log ~ LotArea.log + MSSubClass.avg + Neighborhood, data = .) -> model

model %>% summary
sum(model$residuals ^ 2)


Call:
lm(formula = LotFrontage.log ~ LotArea.log + MSSubClass.avg + 
    Neighborhood, data = .)

Residuals:
     Min       1Q   Median       3Q      Max 
-0.96865 -0.08450  0.00012  0.09137  1.14781 

Coefficients:
                     Estimate Std. Error t value Pr(>|t|)    
(Intercept)         -0.708634   0.108979  -6.502 9.57e-11 ***
LotArea.log          0.316372   0.013373  23.658  < 2e-16 ***
MSSubClass.avg       0.514920   0.024659  20.881  < 2e-16 ***
NeighborhoodBlueste -0.154148   0.078124  -1.973 0.048596 *  
NeighborhoodBrDale  -0.236865   0.059383  -3.989 6.84e-05 ***
NeighborhoodBrkSide -0.179706   0.049795  -3.609 0.000314 ***
NeighborhoodClearCr -0.106477   0.065729  -1.620 0.105377    
NeighborhoodCollgCr -0.173570   0.048374  -3.588 0.000340 ***
NeighborhoodCrawfor -0.090917   0.051179  -1.776 0.075786 .  
NeighborhoodEdwards -0.156170   0.048430  -3.225 0.001278 ** 
NeighborhoodGilbert -0.192760   0.050398  -3.825 0.000134 ***
NeighborhoodIDOTRR  -0.146139   0.05052

[1] 96.20018

In [64]:
df <- df.LotFrontage %>%
select(LotFrontage.log, LotArea.log, MSSubClass, Neighborhood) %>%
group_by(MSSubClass) %>%
mutate(MSSubClass.avg = median(LotFrontage.log)) %>%
group_by(Neighborhood) %>%
mutate(Neighborhood.avg = median(LotFrontage.log))

df %>% 
lm(LotFrontage.log ~ LotArea.log + MSSubClass.avg + Neighborhood.avg, data = .) -> model

model %>% summary
sum(model$residuals ^ 2)


Call:
lm(formula = LotFrontage.log ~ LotArea.log + MSSubClass.avg + 
    Neighborhood.avg, data = .)

Residuals:
     Min       1Q   Median       3Q      Max 
-1.01056 -0.09321  0.00605  0.10197  1.12914 

Coefficients:
                 Estimate Std. Error t value Pr(>|t|)    
(Intercept)      -1.19407    0.07940 -15.039  < 2e-16 ***
LotArea.log       0.30090    0.01299  23.164  < 2e-16 ***
MSSubClass.avg    0.45987    0.02243  20.500  < 2e-16 ***
Neighborhood.avg  0.17439    0.02155   8.092 9.17e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.2045 on 2429 degrees of freedom
Multiple R-squared:  0.6736,	Adjusted R-squared:  0.6732 
F-statistic:  1671 on 3 and 2429 DF,  p-value: < 2.2e-16


[1] 101.6029

In [67]:
df.LotFrontage %>% select(Neighborhood, LotFrontage.log) %>%
group_by(Neighborhood) %>%
summarise(n = n(), median = median(LotFrontage.log)) %>%
mutate(freq = (n / sum(n)) * 100) %>%
arrange(median)

Neighborhood,n,median,freq
BrDale,30,3.044522,1.2330456
MeadowV,33,3.044522,1.3563502
Blueste,10,3.178054,0.4110152
NPkVill,21,3.178054,0.8631319
Blmngtn,20,3.761200,0.8220304
BrkSide,95,3.931826,3.9046445
IDOTRR,87,4.094345,3.5758323
OldTown,229,4.094345,9.4122483
StoneBr,46,4.094345,1.8906700
SWISU,44,4.094345,1.8084669


In [ ]:
df <- df.LotFrontage %>%
select(Neighborhood, LotFrontage.log) %>%
group_by(Neighborhood) %>%
mutate(n = n(), median = median(LotFrontage.log)) %>%
mutate(
    Neighborhood2 = case_when(
        median < 4.1 ~ 'A',
        MSSubClass %in% c('60', '70', '75', '80', '85', '90') ~ 'B',
        TRUE ~ 'C'
)) 

df %>%
select(MSSubClass2, MSSubClass, LotFrontage.log) %>%
group_by(MSSubClass2) %>%
summarise(n = n(), median = median(LotFrontage.log))

df %>% 
lm(LotFrontage.log ~ LotArea.log + MSSubClass2, data = .) -> model

model %>% summary
sum(model$residuals ^ 2)